In [1]:
import pandas as pd
import nltk


In [4]:
books_data = pd.read_csv('./cleaned_data/cleaned_books.csv')
books_data.shape

(141254, 5)

In [7]:
ratings_data = pd.read_csv('./cleaned_data/cleaned_ratings.csv')
ratings_data

,title,price,helpfulness,score,summary,text
0,Its Only Art If Its Well Hung!,NaN,7/7,4.0,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,NaN,10/10,5.0,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,NaN,10/11,5.0,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,NaN,7/7,4.0,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,NaN,3/3,4.0,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...
2193369,The Idea of History,NaN,14/19,4.0,Difficult,"This is an extremely difficult book to digest,..."
2193370,The Idea of History,NaN,1/1,4.0,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2193371,The Idea of History,NaN,0/0,4.0,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2193372,The Idea of History,NaN,1/11,5.0,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


In [13]:
# adapted from https://monkeylearn.com/blog/text-cleaning/
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
tqdm.pandas()

sub = r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"
stemmer = PorterStemmer()
lemmer = WordNetLemmatizer()

stopwords = stopwords.words('english')



def clean_text(sentence):
    #lowercase
    sentence = str(sentence)
    sentence = sentence.lower()
    
    #remove weird characters
    sentence = re.sub(sub, "",sentence)
 
    #remove stopwords
    sentence = " ".join([word for word in sentence.split() if word not in (stopwords)])
    
    #stem words -> Try if a failure
    #sentence = " ".join([stemmer.stem(word) for word in sentence.split(" ")])
    return sentence


cleaned_text = ratings_data['text'].progress_map(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alxto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 2193374/2193374 [1:01:04<00:00, 598.51it/s] 


In [14]:
cleaned_text.to_csv("./cleaned_data/cleaned_text.csv")

SyntaxError: incomplete input (1414357992.py, line 1)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(ratings_data['text'])

KeyboardInterrupt: 

In [ ]:
vectorizer.vocabulary_